In [4]:
#import of packages
import networkx as nx
import matplotlib.pyplot as plt
import random  
import numpy as np
import sys
from  fractions import Fraction
import timeit
from time import sleep
from collections import defaultdict
import heapq as heap
from tqdm import tqdm
from community import community_louvain
import collections 
import matplotlib.cm as cm
import statistics

from IPython.display import HTML, display
from itertools import chain, combinations
from itertools import islice
from collections import Counter, defaultdict
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)
warnings.filterwarnings("ignore", category=FutureWarning)
from joblib import Parallel, delayed
import csv
from pprint import pprint
import math
from operator import itemgetter
import itertools
import scipy
from scipy import stats
from IPython.core.magic import register_line_magic
from IPython.display import HTML, display
import json
import powerlaw
from decimal import Decimal

In [17]:
def get_communities(part,G): # returns the number of communities in G given the partition part
    communities = []
    for i in G.nodes():
        if part[i] not in communities:
            communities.append(part[i])
    return len(communities)

In [19]:
# genarates a WRR of graph G, starting from any node in S.
# takes as input the negative seed nodes S_N, the precomputed paths through which S_N might propaget its information (used to boost the performance)
def construct_WRR(G,S,p,S_N,paths):
    max_depth= float('inf') 
    WRR=set()
    A,s,m,original_node,copies,ancestors_u,descendants_u,contribution,other=[],{},{},{},defaultdict(lambda: 0),defaultdict(lambda: set()),defaultdict(lambda: set()),{},defaultdict(lambda: 0)
    r= random.choices(list(S),weights=p,k=1)[0]
    S_N_WRR=set()
    WRR.add(r);A.append((r,0));original_node[r]=r
    prev_v=defaultdict(lambda: [])
    while A:
        v,l = A.pop(0)
        s[v],m[v]=l,0
        if v==r: m[v]=1
        else: # compute mv
            v_original=original_node[v]
            c=prev_v[v]
            m[v]+=G[v_original][original_node[c]]["negative"]*m[c] 
        
        
        if original_node[v] not in S_N:
            if l>=max_depth: continue
                
            parents=list(G.pred[original_node[v]].keys())
            connections=[u for u in parents if random.random()<=G[u][original_node[v]]["negative"]]
        
            for u in connections:
                if u in WRR:
                    if u in descendants_u[v]: continue
                    if  other[u]>9:continue
                    if not any([set(list(descendants_u[v])+[v]).issubset(x) for x in paths[r]]):other[u]+=1
                    #add a copy of the node
                    copies[u]+=1
                    original_node[str(u)+"copy"+str(copies[u])]=u
                    descendants_u[str(u)+"copy"+str(copies[u])].update(list(descendants_u[v])+[v])
                    A.append((str(u)+"copy"+str(copies[u]),s[v]+1))
                    prev_v[str(u)+"copy"+str(copies[u])]=v
                else:
                    #WRR.add_edge(u,v)
                    WRR.add(u)
                    descendants_u[u].update(list(descendants_u[v])+[v])
                    A.append((u,s[v]+1))   
                    original_node[u]= u
                    prev_v[u]=v   
        else:
            S_N_WRR.add(v) 
            max_depth= l
                
        for node in descendants_u[v]:
            ancestors_u[node].update([v])
                
    prob_inf=1-np.prod([1-m[s_n] for s_n in S_N_WRR])     
    if prob_inf!=0:
        max_prob=max([m[s_n] for s_n in S_N_WRR])
        for u in WRR:
            if not str(u).isnumeric() or m[u]< max_prob or u in S_N_WRR: continue
            contribution[u]=1
        return True,r,1,WRR,contribution,original_node,ancestors_u,s,m,S_N_WRR,copies 
    return False,r,0,WRR,None,original_node,ancestors_u,s,m,None,None

# finds set with maximum blocking power. Used in the loop to generate WRR, required for the stopping criterion
def  Max_Coverage_WRS(RRs,roots,k,G,S_N,S,keep,avg_deg):
    S_k=set()
    need_covering=[x for x,r in enumerate(RRs) ]
    num_RRs=len(need_covering)
    count_roots_total=Counter(roots.values())
    coverage={v:0 for v in G.nodes()}
    for j in RRs.keys():
        for  v in RRs[j]:
            coverage[v]+=1/count_roots_total[roots[j]]
    Cov_S_k=0        
    while len(S_k)< k:
        u = max(coverage,key=coverage.get)
        for j in [j for j in need_covering if u in RRs[j]]:
            for  v in set(RRs[j]):
                coverage[v]-=1/count_roots_total[roots[j]]
        need_covering=set(need_covering)-set([j for j in need_covering if u in RRs[j]])
        S_k.add(u)
        Cov_S_k+= coverage[u]
    return S_k,Cov_S_k

def construct_WFR(G,S_N):
    A,S=[],set()
    r= "new"
    A.append(r)
    first=True
    original={r:r}
    path={r:[]}
    while A:
        v = A.pop(0)
        childreen=iter(G[original[v]])
        connections=[u for u in childreen if random.random()<=G[original[v]][u]["negative"]]
        
        for u in connections:
            if (u not in S_N or first)  :
                if u not in S:
                    path[u]=path[v]+[v]
                    A.append(u)
                    S.add(u)
                    original[u]=u 
        first=False                 
    return S,path


# genarates a WFR of graph G, starting from any node in S.
# takes as input the negative seed nodes S_N, and a threshold, the minimum probability of infection required to consider a path
# used to compute which nodes might get infected
def WFRS(G,threshold,S_N):
    f=Fraction(threshold).limit_denominator()
    denom,num=f.denominator,f.numerator
    S=set()
    probs=defaultdict(lambda: 0)
    count=0
    paths=defaultdict(lambda: [])
    for i in range(1000):
        S_new,path=construct_WFR(G,S_N)
        S.update(S_new)
        count+=1
        for s in S_new: probs[s]+=1; paths[s].append(path[s])
        if len(S)==len(G.nodes()): break
    for s in S: probs[s]*=1/count
    S=[s for s in S if probs[s]>=0.01]
    return S,probs,paths

def ipsilon_WRS(e,d):
    return ((2+Decimal(2/3)*e)*((1/d).ln())*(1/(e**2)))


def D_SSA_WRS(G,S,S_N,k,e,d,max_depth,probs,avg_deg,dis,paths):
    
    exp_inf=np.ceil(np.sum([probs[x] for x in S]))
    S_orig=S.copy()
    n=len(S);d=Decimal(1/n);e=Decimal(e)   
    choose=(Decimal(math.factorial(n))/(Decimal(math.factorial(n-k))*Decimal(math.factorial(k))))
    d_prime=d/6/choose

    N_max=8*(Decimal(1-1/math.e)/(2+2*e/3))*(ipsilon_WRS(e,d_prime))*Decimal(n/k)
    t_max=math.ceil((2*N_max/(ipsilon_WRS(e,d/3))).log10()/Decimal(2).log10())
    t=1
    
    Lambda=round(ipsilon_WRS(e,d/(3*t_max)))
    Lambda_1=1+(1+e)*ipsilon_WRS(e,d/(3*t_max))
    
    
    keep,prob_inf_r,contributions,WRRs,original_nodes,RRs,roots,S_N_u,s,m,S_N_WRR,copies={},{},{},{},{},{},{},{},{},{},{},{}
   
    p=[dis[x] for x in S]
    i=0
    
    while len([x for x,v in keep.items() ])< Lambda*2**(t-1):
        keep[i],roots[i],prob_inf_r[i],WRR,contributions[i],original_nodes[i],S_N_u[i],s[i],m[i],S_N_WRR[i],copies[i]=construct_WRR(G,S,p,S_N,paths)
        WRRs[i]=[e for e in WRR].copy()
        RRs[i]=[n for n in WRR].copy()
        i+=1
        
    RRs_kept=[x for x in keep ]
    RRs_kept_true=[x for x in keep if keep[x]]
    
    while len(RRs_kept)< N_max:
        RRs_kept=[x for x in keep]
        RRs_kept_true=[x for x in keep if keep[x]]
        keep_t,prob_inf_r_t,contributions_t,WRRs_t,original_nodes_t,RRs_t,roots_t,S_N_u_t,s_t,m_t,S_N_WRR_t,copies_t={},{},{},{},{},{},{},{},{},{},{},{}
        i=len(keep) 
        while len([x for x,v in keep_t.items()])< Lambda*2**(t-1):
            keep_t[i],roots_t[i],prob_inf_r_t[i],WRR_t,contributions_t[i],original_nodes_t[i],S_N_u_t[i],s_t[i],m_t[i],S_N_WRR_t[i],copies_t[i]=construct_WRR(G,S,p,S_N,paths)
            
            WRRs_t[i]=[e for e in WRR_t].copy()
            RRs_t[i]=[n for n in WRR_t].copy()
            i+=1
          
        RRs_t_kept_true=[x for x in keep_t if keep_t[x]]
        RRs_t_kept=[x for x in keep_t]
        S_k,I = Max_Coverage_WRS(RRs,roots,k,G,S_N,S,keep,avg_deg) 
        
        count_roots_total_t=Counter(roots_t.values())
        count_roots_total=Counter(roots.values())
        Cov_S_k_t =np.sum([1/count_roots_total_t[roots_t[i]] if ( any([j in RRs_t[i] for j in S_k]) ) else 0 for i,v in keep_t.items()]) 
        if Cov_S_k_t*len(RRs_t)/n>= Lambda_1:
            I_t= Cov_S_k_t
            e1=Decimal((I/I_t)-1)
            e2=e*np.sqrt(Decimal(n*(1+e))/Decimal(2**(t-1)*I_t))
            e3=e*np.sqrt(Decimal(n*(1+e)*(Decimal(1-1/math.e)-e))/Decimal((1+e/3)*2**(t-1)*Decimal(I_t)))
            e_t=(e1+e2+e1*e2)*(Decimal(1-1/math.e)-e)+Decimal(1-1/math.e)*e3
            
            if e_t<=e :
                Cov_S_k_s=np.sum([1/count_roots_total[roots[i]] if ( any([j in RRs[i] for j in S_N]) ) else 0 for i,v in keep.items()])
                if Cov_S_k_s*len(RRs)/n>=Lambda_1: 
                    return keep,contributions,WRRs,original_nodes,roots,RRs,S_N_u,s,m,S_N_WRR,copies,prob_inf_r,e_t 
                Cov_S_t_k_s=np.sum([1/count_roots_total_t[roots_t[i]] if ( any([j in RRs_t[i] for j in S_N]) ) else 0 for i,v in keep_t.items()])
                
                if Cov_S_k_s*len(RRs)/n+Cov_S_t_k_s*len(RRs_t)/n>=Lambda_1:
                    contributions.update(contributions_t)
                    WRRs.update(WRRs_t)
                    original_nodes.update(original_nodes_t)
                    RRs.update(RRs_t)

                    roots.update(roots_t)
                    S_N_u.update(S_N_u_t)
                    s.update(s_t)
                    m.update(m_t)
                    copies.update(copies_t)
                    keep.update(keep_t)
                    S_N_WRR.update(S_N_WRR_t)
                    prob_inf_r.update(prob_inf_r_t)
                    return keep,contributions,WRRs,original_nodes,roots,RRs,S_N_u,s,m,S_N_WRR,copies,prob_inf_r,e_t
        
        t+=1
        
        contributions.update(contributions_t)
        WRRs.update(WRRs_t)
        original_nodes.update(original_nodes_t)
        RRs.update(RRs_t)
       
        roots.update(roots_t)
        S_N_u.update(S_N_u_t)
        s.update(s_t)
        m.update(m_t)
        copies.update(copies_t)
        keep.update(keep_t)
        S_N_WRR.update(S_N_WRR_t)
        prob_inf_r.update(prob_inf_r_t)
    return keep,contributions,WRRs,original_nodes,roots,RRs,S_N_u,s,m,S_N_WRR,copies,prob_inf_r,e_t



#this is the WRRS method proposed in the paper
def WRS_method(G,S_N,k,partition,threshold,min_inf):
    avg_deg=sum([G.out_degree(x) for x in G.nodes()])/len(G.nodes())
    probs={}
    G_extended=G.copy()
    G_extended.add_edges_from([("new",x) for x in S_N])
    for x in S_N:G_extended["new"][x]["negative"]=1
    S,probs,paths=WFRS(G_extended,threshold,S_N)
    paths_blocked=defaultdict(lambda: 0)
    S=list(set(S)-set(S_N+["new"]))
    d={x:probs[x] for x in S}
    paths={x:paths[x] for x in S}
    
    S_P=[]
    e=0.1
    keep,contributions,WRRs,original_nodes,roots_all,RRs,ancestors_u,s,m,S_N_WRR,copies,prob_inf_r,e_new= D_SSA_WRS(G,S,S_N,k,e,1/len(G),partition,probs,avg_deg,d,paths)
    del WRRs, ancestors_u,S_N_WRR,copies,prob_inf_r,e_new
    
    WRRs_to_keep=[i for i,b in enumerate(keep.values()) if b]
    WRRs_to_remove=[i for i,b in enumerate(keep.values()) if not b]
    del keep
    
    num_com=get_communities(partition,G)
    roots={key:roots_all[key] for key in WRRs_to_keep}
    count_roots=Counter(roots.values())
    count_roots_total=Counter(roots_all.values())
    
    
    RRs_copies=RRs.copy()
    prob_infections={x:count_roots[x]/count_roots_total[x] for x in count_roots.keys()}
    current_prob_infections=defaultdict(lambda: 0)
    inform_power=defaultdict(lambda: 0)
      
    initial_infections={}
    
    for x in count_roots.keys():
        current_prob_infections[x]=count_roots[x]/count_roots_total[x]
        if partition[x] not in initial_infections.keys():initial_infections[partition[x]]=0
        initial_infections[partition[x]]+=prob_infections[x]
    
    expected_inf=initial_infections.copy()
    communities_to_save=[x for x in initial_infections.keys() if initial_infections[x]>0]
    Gcc = sorted(nx.connected_components(G.to_undirected()), key=len, reverse=True)
    G_sub = G.subgraph(Gcc[0])
    
    com_g_sub=set([partition[x] for x in G_sub.nodes()])
    
    min_help_c=communities_to_save.copy()
    candidates=list(G.nodes())
    maximin_value=0
    first=True
    
    
    maximin_c= {c:(initial_infections[c]-expected_inf[c])/initial_infections[c] for c in communities_to_save }
    total_inform=defaultdict(lambda: 0)
    while len(S_P)< k:
        max_c=[c for c in communities_to_save if expected_inf[c]==max(expected_inf.values())][0]
        no_help=[c for c in communities_to_save if (initial_infections[c]-expected_inf[c])/initial_infections[c]==0]
        need_saving=[c for c in communities_to_save if (initial_infections[c]-expected_inf[c])/initial_infections[c]<=maximin_value*(1+e)+len(no_help)/len(communities_to_save)*e]
        
        if first:
            inf_u={x:expected_inf.copy() for x in candidates}
            candidates_next=set()
            for j in WRRs_to_keep:
                for  key in contributions[j].keys():
                    candidates_next.add(key)
                    inf_u[key][partition[roots[j]]]-=1/count_roots_total[roots[j]]
                    total_inform[key]+=1/count_roots_total[roots[j]]
             
            for j in WRRs_to_remove:
                for  key in RRs[j]:
                    if not str(key).isnumeric(): continue
                    candidates_next.add(key)
                    total_inform[key]+=1/count_roots_total[roots_all[j]]
            candidates=candidates_next
            
                    
        maximin_c= {c:(initial_infections[c]-expected_inf[c])/initial_infections[c] for c in communities_to_save }
                
        maximin_pev= min([(initial_infections[c]-expected_inf[c])/initial_infections[c] for c in communities_to_save ]) 
        
        maximin_u={x: min([(initial_infections[c]-inf_u[x][c])/initial_infections[c] for c in communities_to_save ]) for x in candidates}
        max_value=max([maximin_u[x] for x in maximin_u.keys()])
        u_maximin=max(maximin_u,key=maximin_u.get)
        maximins_u_m={c:(initial_infections[c]-inf_u[u_maximin][c])/initial_infections[c] for c in communities_to_save}
        c_s=[k for k,x in maximins_u_m.items() if x==max_value]
        c=c_s[0]
        max_inf=max([expected_inf[c] for c in communities_to_save])
        c_s=[c for c in communities_to_save if expected_inf[c]==max_inf]
        c=c_s[0]
        exp_total=np.sum([expected_inf[c]  for c in communities_to_save ])
        u_s=[key for key, value in maximin_u.items() if  value>= max_value*(1-e*(1-expected_inf[c]/exp_total)) ]
        
        if len(u_s)>1:
        
            avg_help=np.mean([(initial_infections[c]-expected_inf[c])/initial_infections[c] for c in communities_to_save ])
            
            need_saving_2=[c for c in communities_to_save if (initial_infections[c]-expected_inf[c])/initial_infections[c]<=avg_help*(1+e)]
            tot={x:np.sum([(expected_inf[c]-inf_u[x][c])  for c in communities_to_save ]) for x in u_s} 
            
            if len(no_help)>0:
                inf_need_saving={x:np.sum([(expected_inf[c]-inf_u[x][c])  for c in no_help]) for x in u_s}
                exp_no_help=np.sum([expected_inf[c] for c in no_help])
                u_s=[x for x in u_s if inf_need_saving[x]>0 or exp_no_help< e*exp_total ]
                dis_c=[x for x in no_help if x not in com_g_sub]
                if len(dis_c)>0:
                    inf_need_saving={x:np.sum([(expected_inf[c]-inf_u[x][c])  for c in dis_c]) for x in u_s}
                    u_s=[x for x in u_s if inf_need_saving[x]>0]
            else: inf_need_saving={x:1 for x in u_s}  
                
              
            inf_need_saving_2={x:np.sum([(expected_inf[c]-inf_u[x][c])*(avg_help-maximin_c[c])   for c in need_saving_2]) for x in u_s }
            

            max_saved=max(inf_need_saving_2.values())
            exp_i=np.sum([expected_inf[c]  for c in need_saving_2])
            init_i=np.sum([initial_infections[c]  for c in need_saving_2])
            options_u=[key for key, value in inf_need_saving_2.items() if  value >= (1-e)*max_saved-max(init_i-exp_i,0)*avg_help]
            
            
            inf_need_saving={x:np.sum([max(expected_inf[c]-inf_u[x][c],0) for c in need_saving]) for x in options_u}
            max_saved_tot=max([inf_need_saving[x] for x in options_u])
            options_u_2=[key for key, value in inf_need_saving.items() if  value >= (1-e)*max_saved_tot-e*max(init_i-exp_i,0) ]
            
            if max([tot[x] for x in options_u_2])< max(tot.values())*(1-e):
                sav_tot={x:np.sum([inf_u[x][c] for c in communities_to_save])  for x in options_u_2}
                opts=[x for x,v in sav_tot.items() if v<= min(sav_tot.values())*(1+e)]
                sav_u={x:np.sum([current_prob_infections[v] for v in list(G.successors(x))+[x] ]) for x in opts}
            else:    
                sav_u={x:np.sum([current_prob_infections[v] for v in list(G.successors(x))+[x] ]) for x in options_u_2}
            u=max(sav_u,key=sav_u.get)
                                                                                                                                  
        else: u= u_s[0]
            
        
        
        expected_inf=inf_u[u].copy()
        helps_u={c:(initial_infections[c]-expected_inf[c])/initial_infections[c] for c in communities_to_save}
        min_help=min(helps_u.values())
        maximin_value=min_help
        S_P.append(u)
        
        first=False
        candidates-=set([u])
        to_remove=set()
        for j in [j for j in WRRs_to_keep if u in contributions[j] ]:
            current_prob_infections[roots[j]]-=1/count_roots_total[roots[j]]
            current_prob_infections[roots[j]]=max(0,current_prob_infections[roots[j]]) 
            for w in set(candidates)-set(contributions[j].keys()):
                inf_u[w][partition[roots[j]]]-=1/count_roots_total[roots[j]]
            for w in set(candidates)&set(contributions[j].keys()):
                total_inform[w]-=1/count_roots_total[roots[j]]
        WRRs_to_keep=list(set(WRRs_to_keep)-set([j for j in WRRs_to_keep if u in contributions[j]]))
            
               
    return S_P

In [20]:
#code to set probability of the edges
def set_weight_degree(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= 1/G.in_degree(v)
    return G

def set_weight_trivalency(G,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= random.choice([0.1,0.01,0.001])
    return G

def set_weight_value(G,val,p_n):
    for u,v in G.edges():
        G[u][v][p_n]= round(val,3)
    return G

def set_weight_average(G,p_n):
    avg=sum([G.out_degree(x) for x in G.nodes()])/len(G.nodes())
    for u,v in G.edges():
        G[u][v][p_n]= 1/avg
    return G

def set_weight_comm(G,inter,intra,p_n):
    for u,v in G.edges():
        if G.nodes[u]['community']== G.nodes[v]['community']:
            G[u][v][p_n]= inter
        else:
            G[u][v][p_n]= intra
    return G

In [21]:
network=nx.erdos_renyi_graph(1000,0.4,directed=True)
# the network nodes need to have a lavel commuity defining to which community they belong to
partition=community_louvain.best_partition(network.to_undirected())
for node in network.nodes():
    network.nodes[node]['community']=partition[node]
    
# the network shall also have allocated to each edge the probability of infection        
network=set_weight_degree(network,"positive")
network=set_weight_degree(network,"negative")

k_p=100
k_n=50
S_N=random.choices([v for v in list(network.nodes())],k= k_n)
S_P=WRS_method(network,S_N,k_p,partition,0.01,0)
print(S_P)

[950, 280, 978, 646, 353, 274, 44, 391, 803, 625, 88, 688, 309, 412, 902, 727, 622, 711, 626, 181, 77, 317, 163, 520, 133, 676, 748, 427, 549, 988, 304, 679, 52, 225, 377, 852, 351, 883, 430, 197, 862, 456, 737, 419, 126, 917, 34, 93, 998, 618, 70, 990, 838, 510, 999, 389, 818, 316, 481, 546, 712, 958, 761, 434, 757, 65, 318, 293, 143, 4, 184, 582, 403, 463, 261, 657, 339, 796, 884, 234, 611, 431, 69, 249, 313, 485, 522, 121, 684, 266, 776, 794, 514, 357, 22, 321, 277, 756, 360, 152]
